In [1]:
import os

import pandas as pd

In [2]:
urls = {
    "2023": "https://download.inep.gov.br/dados_abertos/microdados_avaliacao_da_alfabetizacao_2023.zip",
    "2024": "https://download.inep.gov.br/dados_abertos/microdados_avaliacao_da_alfabetizacao_2024.zip",
}

# Diretório base
input_dir = os.path.join("models", "br_inep_avaliacao_alfabetizacao", "input")
os.makedirs(input_dir, exist_ok=True)

# Pastas de entrada
input_paths = {
    "2023": os.path.join(input_dir, "ica_23"),
    "2024": os.path.join(input_dir, "ica_24"),
}
for path in input_paths.values():
    os.makedirs(path, exist_ok=True)

# Pastas de saída
output_dir = os.path.join(
    "models", "br_inep_avaliacao_alfabetizacao", "output"
)
os.makedirs(output_dir, exist_ok=True)


# # Loop de download e extração
# for ano, url in urls.items():
#     filename = url.split("/")[-1]
#     filepath = os.path.join(input_dir, filename)

#     # Baixar arquivo
#     print(f"Baixando {filename}...")
#     response = requests.get(url, stream=True)
#     response.raise_for_status()

#     with open(filepath, "wb") as f:
#         for chunk in response.iter_content(chunk_size=8192):
#             f.write(chunk)

#     print(f"{filename} baixado com sucesso.")

#     # Extrair no diretório certo
#     print(f"Extraindo {filename} em {input_paths[ano]}...")
#     with zipfile.ZipFile(filepath, "r") as zip_ref:
#         zip_ref.extractall(input_paths[ano])

#     print(f"{filename} extraído em {input_paths[ano]}.\n")

# Alunos

In [6]:
# Arquivos por ano
arquivo_23 = os.path.join(input_dir, "ica_23/DADOS/TS_ALUNO.csv")
arquivo_24 = os.path.join(input_dir, "ica_24/DADOS/TS_ALUNO.csv")


arquivos = {"2023": arquivo_23, "2024": arquivo_24}

# Lê apenas os cabeçalhos
df_23 = pd.read_csv(arquivo_23, sep=";", encoding="latin1", nrows=0)
df_24 = pd.read_csv(arquivo_24, sep=";", encoding="latin1", nrows=0)

# Colunas
print(df_23.columns.tolist())
print(df_24.columns.tolist())

['NU_ANO_AVALIACAO', 'CO_UF', 'SG_UF', 'ID_ALUNO', 'TP_SERIE', 'ID_ESCOLA', 'TP_DEPENDENCIA', 'CO_MUNICIPIO', 'NO_MUNICIPIO', 'IN_PRESENCA_LP', 'IN_PREENCHIMENTO_LP', 'CO_CADERNO_LP', 'VL_PESO_ALUNO_LP', 'VL_PROFICIENCIA_LP', 'IN_ALFABETIZADO']
['NU_ANO_AVALIACAO', 'CO_UF', 'SG_UF', 'ID_ALUNO', 'TP_SERIE', 'ID_ESCOLA', 'TP_DEPENDENCIA', 'CO_MUNICIPIO', 'NO_MUNICIPIO', 'IN_PRESENCA_LP', 'IN_PREENCHIMENTO_LP', 'CO_CADERNO_LP', 'VL_PESO_ALUNO_LP', 'VL_PROFICIENCIA_LP', 'IN_ALFABETIZADO']


In [7]:
# Renomear colunas
rename_cols = {
    "NU_ANO_AVALIACAO": "ano",
    "CO_UF": "id_regiao",
    "SG_UF": "sigla_uf",
    "TP_SERIE": "serie",
    "TP_DEPENDENCIA": "rede",
    "CO_MUNICIPIO": "id_municipio",
    "IN_PRESENCA_LP": "presenca",
    "IN_PREENCHIMENTO_LP": "preenchimento_caderno",
    "CO_CADERNO_LP": "caderno",
    "VL_PESO_ALUNO_LP": "peso_aluno",
    "VL_PROFICIENCIA_LP": "proficiencia",
    "IN_ALFABETIZADO": "alfabetizado",
}

In [8]:
# Salvar o arquivo final
output_csv = os.path.join(output_dir, "alunos.csv")

col_order = [
    "ano",
    "sigla_uf",
    "id_municipio",
    "id_escola",
    "id_aluno",
    "caderno",
    "serie",
    "rede",
    "presenca",
    "preenchimento_caderno",
    "alfabetizado",
    "proficiencia",
    "peso_aluno",
]

for ano, arquivo in arquivos.items():
    print(f"Processando {ano}...")
    chunks = pd.read_csv(arquivo, sep=";", encoding="latin1", chunksize=500000)
    for i, chunk in enumerate(chunks):
        chunk.rename(columns=rename_cols, inplace=True)
        chunk.columns = [col.lower() for col in chunk.columns]

        cols_drop = ["co_uf", "no_municipio", "id_regiao", "sigla_uf"]
        chunk.drop(
            columns=[c for c in cols_drop if c in chunk.columns], inplace=True
        )

        # adiciona coluna do ano caso não exista
        if "ano" not in chunk.columns:
            chunk["ano"] = ano

        # reordena colunas (mantendo só as que existem)
        cols_final = [c for c in col_order if c in chunk.columns]
        chunk = chunk[cols_final]

        # escreve no CSV final
        if i == 0 and ano == "2023":
            chunk.to_csv(output_csv, sep=",", index=False, mode="w")
        else:
            chunk.to_csv(
                output_csv, sep=",", index=False, mode="a", header=False
            )

        print(f"Chunk {i + 1} do ano {ano} salvo.")

Processando 2023...
Chunk 1 do ano 2023 salvo.
Chunk 2 do ano 2023 salvo.
Chunk 3 do ano 2023 salvo.
Chunk 4 do ano 2023 salvo.
Processando 2024...
Chunk 1 do ano 2024 salvo.
Chunk 2 do ano 2024 salvo.
Chunk 3 do ano 2024 salvo.
Chunk 4 do ano 2024 salvo.
Chunk 5 do ano 2024 salvo.


# Municipio

In [4]:
# Arquivos por ano
municipio_23 = os.path.join(input_dir, "ica_23/DADOS/TS_MUNICIPIO.csv")
municipio_24 = os.path.join(input_dir, "ica_24/DADOS/TS_MUNICIPIO.csv")

arquivos = {"2023": municipio_23, "2024": municipio_24}

# Lê apenas os cabeçalhos
df_23 = pd.read_csv(municipio_23, sep=";", encoding="latin1", nrows=0)
df_24 = pd.read_csv(municipio_24, sep=";", encoding="latin1", nrows=0)

# Colunas
print(df_23.columns.tolist())
print(df_24.columns.tolist())

['NU_ANO_AVALIACAO', 'CO_UF', 'SG_UF', 'CO_MUNICIPIO', 'NO_MUNICIPIO', 'TP_SERIE', 'ID_TIPO_REDE', 'PC_ALUNO_ALFABETIZADO', 'VL_MEDIA_LP']
['NU_ANO_AVALIACAO', 'CO_UF', 'SG_UF', 'CO_MUNICIPIO', 'NO_MUNICIPIO', 'TP_SERIE', 'ID_TIPO_REDE', 'PC_ALUNO_ALFABETIZADO', 'VL_MEDIA_LP', 'PC_ALUNO_NIVEL_0_LP', 'PC_ALUNO_NIVEL_1_LP', 'PC_ALUNO_NIVEL_2_LP', 'PC_ALUNO_NIVEL_3_LP', 'PC_ALUNO_NIVEL_4_LP', 'PC_ALUNO_NIVEL_5_LP', 'PC_ALUNO_NIVEL_6_LP', 'PC_ALUNO_NIVEL_7_LP', 'PC_ALUNO_NIVEL_8_LP']


In [5]:
# Renomear colunas
rename_cols = {
    "NU_ANO_AVALIACAO": "ano",
    "SG_UF": "sigla_uf",
    "TP_SERIE": "serie",
    "ID_TIPO_REDE": "rede",
    "CO_MUNICIPIO": "id_municipio",
    "PC_ALUNO_ALFABETIZADO": "taxa_alfabetizacao",
    "VL_MEDIA_LP": "media_portugues",
}


# Dicionário de renomeação das colunas de nível LP
rename_niveis = {
    f"PC_ALUNO_NIVEL_{i}_LP": f"proporcao_aluno_nivel_{i}" for i in range(9)
}

# Combina os dois dicionários
rename_cols.update(rename_niveis)

In [6]:
# Lê os dados completos (não só cabeçalho)
df_23 = pd.read_csv(municipio_23, sep=";", encoding="latin1")
df_24 = pd.read_csv(municipio_24, sep=";", encoding="latin1")

# Renomeia as colunas
df_23.rename(columns=rename_cols, inplace=True)
df_24.rename(columns=rename_cols, inplace=True)

# Mostra as primeiras linhas
print("DF 2023:")
print(df_23.head())

print("\nDF 2024:")
print(df_24.head())

DF 2023:
    ano  CO_UF sigla_uf  id_municipio           NO_MUNICIPIO  serie  rede  \
0  2023     11       RO       1100015  Alta Floresta D'Oeste      2     5   
1  2023     11       RO       1100015  Alta Floresta D'Oeste      2     3   
2  2023     11       RO       1100023              Ariquemes      2     3   
3  2023     11       RO       1100023              Ariquemes      2     5   
4  2023     11       RO       1100031                 Cabixi      2     5   

   taxa_alfabetizacao  media_portugues  
0               64.55         758.3304  
1               64.55         758.3304  
2               62.30         757.0999  
3               62.30         757.0999  
4               69.10         767.8763  

DF 2024:
    ano  CO_UF sigla_uf  id_municipio           NO_MUNICIPIO  serie  rede  \
0  2024     11       RO       1100015  Alta Floresta D'Oeste      2     3   
1  2024     11       RO       1100015  Alta Floresta D'Oeste      2     5   
2  2024     11       RO       1100023    

In [7]:
colunas_desejadas = [
    "ano",
    "id_municipio",
    "serie",
    "rede",
    "taxa_alfabetizacao",
    "media_portugues",
] + [f"proporcao_aluno_nivel_{i}" for i in range(9)]

# Garantir que todas existam
for col in colunas_desejadas:
    if col not in df_23.columns:
        df_23[col] = pd.NA
    if col not in df_24.columns:
        df_24[col] = pd.NA

# Seleciona apenas as colunas desejadas
df_23 = df_23[colunas_desejadas]
df_24 = df_24[colunas_desejadas]

# Une os dois dataframes
df_final = pd.concat([df_23, df_24], ignore_index=True)

# Prévia
print(df_final.head())

/tmp/ipykernel_1973/178038299.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_23, df_24], ignore_index=True)


    ano  id_municipio  serie  rede  taxa_alfabetizacao  media_portugues  \
0  2023       1100015      2     5               64.55         758.3304   
1  2023       1100015      2     3               64.55         758.3304   
2  2023       1100023      2     3               62.30         757.0999   
3  2023       1100023      2     5               62.30         757.0999   
4  2023       1100031      2     5               69.10         767.8763   

   proporcao_aluno_nivel_0  proporcao_aluno_nivel_1  proporcao_aluno_nivel_2  \
0                      NaN                      NaN                      NaN   
1                      NaN                      NaN                      NaN   
2                      NaN                      NaN                      NaN   
3                      NaN                      NaN                      NaN   
4                      NaN                      NaN                      NaN   

   proporcao_aluno_nivel_3  proporcao_aluno_nivel_4  proporcao_aluno

In [8]:
df_final

,ano,id_municipio,serie,rede,taxa_alfabetizacao,media_portugues,proporcao_aluno_nivel_0,proporcao_aluno_nivel_1,proporcao_aluno_nivel_2,proporcao_aluno_nivel_3,proporcao_aluno_nivel_4,proporcao_aluno_nivel_5,proporcao_aluno_nivel_6,proporcao_aluno_nivel_7,proporcao_aluno_nivel_8
0,2023,1100015,2,5,64.55,758.3304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023,1100015,2,3,64.55,758.3304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023,1100023,2,3,62.30,757.0999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023,1100023,2,5,62.30,757.0999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023,1100031,2,5,69.10,767.8763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23990,2024,5222203,2,5,78.69,763.2600,0.00,0.00,6.56,9.84,14.75,27.87,31.15,6.56,3.28
23991,2024,5222302,2,3,79.46,764.4800,3.38,2.94,1.69,4.71,13.77,35.38,19.07,16.13,2.94
23992,2024,5222302,2,5,79.46,764.4800,3.38,2.94,1.69,4.71,13.77,35.38,19.07,16.13,2.94
23993,2024,5300108,2,2,59.13,743.0100,2.44,4.67,7.66,9.63,25.61,33.05,13.13,3.10,0.71


In [9]:
output_csv = os.path.join(output_dir, "municipio.csv")
df_final.to_csv(output_csv, index=False, sep=",")

# UF

In [ ]:
# Arquivos por ano
uf_23 = os.path.join(input_dir, "ica_23/DADOS/TS_ESTADO.csv")
uf_24 = os.path.join(input_dir, "ica_24/DADOS/TS_ESTADO.csv")

arquivos = {"2023": uf_23, "2024": uf_24}

# Lê apenas os cabeçalhos
df_23 = pd.read_csv(uf_23, sep=";", encoding="latin1", nrows=0)
df_24 = pd.read_csv(uf_24, sep=";", encoding="latin1", nrows=0)

# Colunas
cols_23 = df_23.columns.tolist()
cols_24 = df_24.columns.tolist()

print("Colunas 2023:", cols_23)
print("Colunas 2024:", cols_24)

# Conferir se são iguais
if cols_23 == cols_24:
    print("Os arquivos têm exatamente as mesmas colunas.")
else:
    print("Os arquivos têm colunas diferentes.")
    print(
        "Colunas presentes em 2023 mas não em 2024:",
        list(set(cols_23) - set(cols_24)),
    )
    print(
        "Colunas presentes em 2024 mas não em 2023:",
        list(set(cols_24) - set(cols_23)),
    )

Colunas 2023: ['NU_ANO_AVALIACAO', 'CO_UF', 'SG_UF', 'TP_SERIE', 'ID_TIPO_REDE', 'PC_ALUNO_ALFABETIZADO', 'VL_MEDIA_LP']
Colunas 2024: ['NU_ANO_AVALIACAO', 'CO_UF', 'SG_UF', 'TP_SERIE', 'ID_TIPO_REDE', 'PC_ALUNO_ALFABETIZADO', 'VL_MEDIA_LP', 'PC_ALUNO_NIVEL_0_LP', 'PC_ALUNO_NIVEL_1_LP', 'PC_ALUNO_NIVEL_2_LP', 'PC_ALUNO_NIVEL_3_LP', 'PC_ALUNO_NIVEL_4_LP', 'PC_ALUNO_NIVEL_5_LP', 'PC_ALUNO_NIVEL_6_LP', 'PC_ALUNO_NIVEL_7_LP', 'PC_ALUNO_NIVEL_8_LP']
Os arquivos têm colunas diferentes.
Colunas presentes em 2023 mas não em 2024: []
Colunas presentes em 2024 mas não em 2023: ['PC_ALUNO_NIVEL_0_LP', 'PC_ALUNO_NIVEL_1_LP', 'PC_ALUNO_NIVEL_5_LP', 'PC_ALUNO_NIVEL_4_LP', 'PC_ALUNO_NIVEL_2_LP', 'PC_ALUNO_NIVEL_3_LP', 'PC_ALUNO_NIVEL_8_LP', 'PC_ALUNO_NIVEL_6_LP', 'PC_ALUNO_NIVEL_7_LP']


In [ ]:
rename_cols = {
    "NU_ANO_AVALIACAO": "ano",
    "SG_UF": "sigla_uf",
    "TP_SERIE": "serie",
    "ID_TIPO_REDE": "rede",
    "PC_ALUNO_ALFABETIZADO": "taxa_alfabetizacao",
    "VL_MEDIA_LP": "media_portugues",
}

In [ ]:
# Salvar o arquivo final
output_csv = os.path.join(output_dir, "uf.csv")

# Colunas finais na ordem desejada
final_cols = [
    "ano",
    "sigla_uf",
    "serie",
    "rede",
    "taxa_alfabetizacao",
    "media_portugues",
    "pc_aluno_nivel_0_lp",
    "pc_aluno_nivel_1_lp",
    "pc_aluno_nivel_2_lp",
    "pc_aluno_nivel_3_lp",
    "pc_aluno_nivel_4_lp",
    "pc_aluno_nivel_5_lp",
    "pc_aluno_nivel_6_lp",
    "pc_aluno_nivel_7_lp",
    "pc_aluno_nivel_8_lp",
]

# Lista para acumular os DataFrames
dfs = []

for ano, arquivo in arquivos.items():
    print(f"Processando {ano}...")

    # Lê o arquivo inteiro
    df = pd.read_csv(arquivo, sep=";", encoding="latin1")

    # Renomeia colunas
    df.rename(columns=rename_cols, inplace=True)
    df.columns = [col.lower() for col in df.columns]

    # Garante que a coluna "ano" exista
    if "ano" not in df.columns:
        df["ano"] = ano

    # Mantém só as colunas desejadas
    for col in final_cols:
        if col not in df.columns:
            df[col] = pd.NA

    df = df[final_cols]
    dfs.append(df)

# Concatena todos os anos
df_final = pd.concat(dfs, ignore_index=True)

# Salva no CSV final
df_final.to_csv(output_csv, index=False, sep=",")

# Prévia das primeiras 10 linhas
print(df_final.head(10))

Processando 2023...
Processando 2024...
    ano sigla_uf  serie  rede  taxa_alfabetizacao  media_portugues  \
0  2023       RO      2     2               58.65         751.4731   
1  2023       RO      2     3               65.17         760.1971   
2  2023       RO      2     5               64.60         759.4357   
3  2023       AM      2     3               49.20         733.6637   
4  2023       AM      2     5               52.20         736.4687   
5  2023       AM      2     2               62.63         746.2058   
6  2023       PA      2     5               48.38         733.2877   
7  2023       PA      2     3               47.71         732.6697   
8  2023       PA      2     2               57.07         741.3548   
9  2023       AP      2     2               40.70         731.4349   

   pc_aluno_nivel_0_lp  pc_aluno_nivel_1_lp  pc_aluno_nivel_2_lp  \
0                  NaN                  NaN                  NaN   
1                  NaN                  NaN          

/tmp/ipykernel_1649/778779460.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat(dfs, ignore_index=True)
